# Google Gemini API Integration with Instantly

This notebook demonstrates how to use Google's Gemini API through the Instantly library. We'll explore various features including text generation, chat conversations, multimodal inputs, and model information.

## Authentication Setup

First, let's set up our environment and initialize the Google AI client using the Instantly library.

In [ ]:
import os
from instantly import GoogleAIClient

# Initialize the client with your API key
client = GoogleAIClient(api_key=os.environ.get("GEMINI_API_KEY"))

# You can also set the API key in your environment:
# os.environ["GEMINI_API_KEY"] = "your-api-key-here"

## Standard Content Generation

Let's start with basic text generation using the `generateContent` endpoint. This is best for non-interactive tasks where you can wait for the complete response.

In [ ]:
# Simple text generation with default parameters
response = client.chat_completion(
    model="gemini-2.5-pro",
    messages=[
        {"role": "user", "content": "Explain how AI works in a single paragraph."}
    ]
)

print("Response:", response["choices"][0]["message"]["content"])

# Text generation with customized parameters
response = client.chat_completion(
    model="gemini-2.5-pro",
    messages=[
        {"role": "user", "content": "Write a creative story about a robot learning to paint."}
    ],
    temperature=0.9,  # Higher temperature for more creative output
    thinking_config={"thinking_budget": -1}  # No limit on thinking time
)

print("\nCreative story:", response["choices"][0]["message"]["content"])

## Streaming Content Generation

For interactive applications like chatbots, the streaming API provides a better user experience by returning content chunks as they're generated.

In [ ]:
# Stream content generation
stream = client.stream_chat_completion(
    model="gemini-2.5-pro",
    messages=[
        {"role": "user", "content": "Write a poem about coding, one line at a time."}
    ]
)

print("Streaming response:")
for chunk in stream:
    if "content" in chunk["choices"][0]["delta"]:
        print(chunk["choices"][0]["delta"]["content"], end="")

## Chat Conversations

For multi-turn conversations, we need to maintain the chat history by appending each message to the conversation.

In [ ]:
# Initialize conversation history
conversation = [
    {"role": "system", "content": "You are a helpful AI assistant who likes to explain complex topics simply."},
    {"role": "user", "content": "Hi! Can you help me understand quantum computing?"}
]

# Get first response
response = client.chat_completion(
    model="gemini-2.5-pro",
    messages=conversation
)
assistant_message = response["choices"][0]["message"]
conversation.append(assistant_message)
print("Assistant:", assistant_message["content"])

# Continue conversation
conversation.append({"role": "user", "content": "Can you give a simple example of what a qubit is?"})
response = client.chat_completion(
    model="gemini-2.5-pro",
    messages=conversation
)
assistant_message = response["choices"][0]["message"]
conversation.append(assistant_message)
print("\nAssistant:", assistant_message["content"])

## Multimodal Inputs

Gemini can process both text and images in a single prompt. Here's how to send an image along with text:

In [ ]:
import base64
from PIL import Image
from io import BytesIO

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Example of analyzing an image (you would need a real image file)
"""
image_path = "path/to/your/image.jpg"
image_data = encode_image(image_path)

response = client.chat_completion(
    model="gemini-2.5-pro",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "inline_data",
                    "mime_type": "image/jpeg",
                    "data": image_data
                },
                {
                    "type": "text",
                    "text": "What can you see in this image?"
                }
            ]
        }
    ]
)

print("Analysis:", response["choices"][0]["message"]["content"])
"""

## Model Information

Let's explore how to get information about available models and their capabilities.

In [ ]:
# List available models
models = client.list_models()
print("Available models:")
for model in models["models"]:
    print(f"\nModel: {model['name']}")
    if "description" in model:
        print(f"Description: {model['description']}")
    print(f"Token limits: Input={model.get('inputTokenLimit', 'N/A')}, Output={model.get('outputTokenLimit', 'N/A')}")
    print(f"Supported methods: {', '.join(model.get('supportedGenerationMethods', []))}")
    print(f"Temperature range: 0.0 to {model.get('maxTemperature', 1.0)}")

## Next Steps

Here are some ideas for further exploration:
- Experiment with different model parameters like temperature and top_p
- Try more complex multimodal prompts combining text and images
- Implement error handling and retries for production use
- Explore specialized models for tasks like image generation